# Branching

Up until now our repository had only one branch with one commit coming after the other:  
<img src="img/git-branch-1.svg" alt="Linear Git repository" width="150" height="100"/>  

- Commits are depicted here as little boxes with abbreviated hashes.
- Here the branch `master` points to a commit.
- "HEAD" is the current position (remember the recording head of tape recorders?).
- When we talk about branches, we often mean all parent commits, not only the commit pointed to.

Software development is often not linear:

- We typically need at least one version of the code to "work" (to compile, to give expected results, ...).
- At the same time we work on new features, often several features concurrently.
  Often they are unfinished.
- We need to be able to separate different lines of work really well.

The strength of version control is that it permits to **isolate
different tracks of work**, which can later be merged to create a composite
version that contains all changes:

<img src="img/git-collaborative.svg" alt="Isolated tracks of work" width="300" height="200"/>

- We see branching points and merging points.
- Main line development is often called `master` or `main`.
- Other than this convention there is nothing special about `master` or `main`, it is just a branch.
- Commits form a directed acyclic graph (we have left out the arrows to avoid confusion about the time arrow).

A group of commits that create a single narrative are called a **branch**.
There are different branching strategies, but it is useful to think that a branch
tells the story of a feature, e.g. "fast sequence extraction" or "Python interface" or "fixing bug in
matrix inversion algorithm".

### An alias

We will now define an alias in Git, to be able to nicely visualize branch structure in the terminal without having to remember a long Git command

In [ ]:
%%bash
git config --global alias.graph "log --all --graph --decorate --oneline"

## On which branch are we?

To see where we are (where HEAD points to) use `git branch`:

In [ ]:
%cd recipe

In [ ]:
%%bash
git branch

- This command shows where we are, it does not create a branch.
- There is only `main` and we are on `main` (star represents the `HEAD`).

In the following we will learn how to create branches,
how to switch between them, how to merge branches,
and how to remove them afterwards.

## Creating and working with branches

### create a first branch

Let's create a branch called `experiment` where we add cilantro to `ingredients.txt`.

In [ ]:
%%bash
git branch experiment main     # creates branch "experiment" from main
git checkout experiment        # switch to branch "experiment"                  

- Verify that you are on the `experiment` branch

In [ ]:
%%bash
git branch                   # list all local branches and show on which branch we are

- Then add 2 tbsp cilantro **on top** of the `ingredients.txt`:

In [ ]:
%%writefile ingredients.txt

* 2 tbsp cilantro
* 2 avocados
* 1 lime
* 2 tsp salt
* 1/2 onion

- Stage this and commit it

In [ ]:
%%bash
git add .
git commit -m "let us try with some cilantro"

- Then reduce the amount of cilantro to 1 tbsp, stage and commit again

In [ ]:
%%writefile ingredients.txt

* 1 tbsp cilantro
* 2 avocados
* 1 lime
* 2 tsp salt
* 1/2 onion

In [ ]:
%%bash
git add .
git commit -m "maybe little bit less cilantro"

In [ ]:
%%bash
git graph

- The branch `experiment` is two commits ahead of `main`.
- We commit our changes to this branch.

### create a second branch

Let's create a branch called `less-salt` where we reduce the amount of salt in `ingredients.txt`.  

> **Warning:**  makes sure you are on main branch when you create the less-salt branch

In [ ]:
%%bash
git checkout main
git branch less-salt main

git checkout less-salt

In [ ]:
%%writefile ingredients.txt

* 2 avocados
* 1 lime
* 1 tsp salt
* 1/2 onion

In [ ]:
%%bash
git add ingredients.txt
git commit -m "reduce amount of salt"

In [ ]:
%%bash
git graph


Here is a graphical representation of what we have created:  
  
<img src="img/git-branch-2.svg" alt="Isolated tracks of work" width="300" height="200"/>

- Now switch to `main`.
- Add and commit the following `README.md` to `main`:


In [ ]:
%%bash
git checkout main

In [ ]:
%%writefile README.md

# Guacamole recipe

Used in teaching Git.

In [ ]:
%%bash
git add README.md
git commit -m "draft a readme"

In [ ]:
%%bash
git graph

Here is a graphical representation of what we have created:  
  
<img src="img/git-branch-3.svg" alt="" width="300" height="200"/>

# Merging branches

## Merging branch `experiment`

It turned out that our experiment with cilantro was a good idea.
Our goal now is to merge `experiment` into `main`.

First we make sure we are on the branch we wish to merge into:

In [ ]:
%%bash
git branch

Then we merge `experiment` into `main`:

In [ ]:
%%bash
git merge experiment

<img src="img/git-merge-1.svg" alt="" width="300" height="200"/>

We can verify the result in the terminal:

In [ ]:
%%bash
git graph

What happens internally when you merge two branches is that Git creates a new
commit, attempts to incorporate changes from both branches and records the
state of all files in the new commit. While a regular commit has one parent, a
merge commit has two (or more) parents.

To view the branches that are merged into the current branch we can use the command:

In [ ]:
%%bash
git branch --merged

## Merging branch `less-salt`

We are also happy with the work on the `less-salt` branch. Let us merge that
one, too, into `main`:

First we make sure we are on the branch we wish to merge into:

In [ ]:
%%bash
git branch  # make sure you are on master

Then we merge `less-salt` into `main`:

In [ ]:
%%bash
git merge less-salt

<img src="img/git-merge-2.svg" alt="" width="300" height="200"/>

We can verify the result in the terminal:

In [ ]:
%%bash
git graph

Observe how Git nicely merged the changed amount of salt and the new ingredient **in the same file
without us merging it manually**:

In [ ]:
%%bash
cat ingredients.txt

If the same file is changed in both branches, Git attempts to incorporate both
changes into the merged file. If the changes overlap then the user has to
manually *settle merge conflicts* (we will do that later).

# Deleting branches safely

Both feature branches are merged:

In [ ]:
%%bash
git branch --merged

This means we can delete the branches:

In [ ]:
%%bash
git branch -d experiment less-salt

This is the result:  

<img src="img/git-deleted-branches.svg" alt="" width="300" height="200"/>

Compare in the terminal:

In [ ]:
%%bash
git graph

Comparing figures "Commit graph after merge"
and "Commit graph after merged branches were deleted",
we observe that
only the pointers ("sticky notes") disappeared, not the commits.

> **Note:**  Git will not let you delete a branch which has not been reintegrated unless you
insist using `git branch -D`. Even then your commits will not be lost but you
may have a hard time finding them as there is no branch pointing to them.

# Tags

- A tag is a pointer to a commit but in contrast to a branch it **does not ever move**.
- We use tags to record particular states or milestones of a project at a given
  point in time, like for instance versions (have a look at [semantic versioning](http://semver.org),
  v1.0.3 is easier to understand and remember than 64441c1934def7d91ff0b66af0795749d5f1954a).


- It can be useful to think of branches as sticky notes and of tags as
  [commemorative plaques](https://en.wikipedia.org/wiki/Commemorative_plaque).

Let's add an annotated tag to our current state of the guacamole recipe:

In [ ]:
%%bash
git tag -a nobel-2021 -m "recipe I made for the 2021 Nobel banquet"

To see tags list:

In [ ]:
%%bash
git tag

# Practical advice

## What level of branching complexity is necessary for each project?


### Simple personal projects

- Typically start with just the `main` branch.
- Use branches for unfinished/untested ideas.
- Use branches when you are not sure about a change.
- Use tags to mark important milestones.


### Projects with few persons: you accept things breaking sometimes

- It might be reasonable to commit to the `main` branch and feature branches.


### Projects with few persons: changes are reviewed by others

- The `main` branch is write-protected.
- You create new feature branches for changes.
- Changes are reviewed before they are merged to the `main` branch
  (more about that in the [collaborative Git lesson](https://coderefinery.github.io/git-collaborative/)).


### When you distribute releases

- If you want to patch releases, you probably need release branches.
- The `main` branch and release branches are read-only.
- Many [branching models](https://coderefinery.github.io/git-branch-design/05-branching-models/) exist.

# Summary

```console
$ git branch               # see where we are
$ git branch <name> <src>  # create branch <name> from the branch <src>
$ git branch <name>        # create branch <name> from the current one
$ git checkout <name>      # switch to branch <name>
$ git merge <name>         # merge branch <name> (to current branch)
$ git branch -d <name>     # delete branch <name>
$ git branch -D <name>     # delete unmerged branch
```

Since the following command combo is so frequent:

```console
$ git branch <name>        # create branch <name>
$ git checkout <name>      # switch to branch <name>
```

There is a shortcut for it:

```console
$ git checkout -b <name>   # create branch <name> and switch to it
```

---

[back <](git-basics.ipynb)  [menu](../git-intro.ipynb)  [> next](git-conflict.ipynb)  

---